# Location Designator
This example will show you waht location designator are, how to use them and what they are capable of. 

Location Deisgnator are used to semantically describe locations in the world. You could, for example, create a location designator that describes every position where a robot can be placed without colliding with the environment. Location designator can describe locations for:

   * Visibility 
   * Reachability
   * Occupany 
   * URDF Links (for example a table)

To find locations that fit the given constrains, location designator create Costmaps. Costmaps are a 2D distribution that have a value greater than 0 for every position that fits the costmap criteria.

Location designator work similar to other designator, meaning you have to create a location designator description which describes the location. This description can then be resolved to the actual 6D pose on runtime.

## Occupancy

We will start with a simple location designator that describes a location where the robot can be placed without colliding with the environment. To do this we need a BulletWorld since the costmaps are mostly created from the current state of the BulletWorld. 

In [1]:
from pycram.bullet_world import BulletWorld, Object
from pycram.enums import ObjectType
from pycram.pose import Pose

world = BulletWorld()
kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")

Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Scalar element defined multiple times: limit
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Scalar element defined multiple times: limit


In [11]:
world.exit()

Next up we will create the location designator description, the ```CostmapLocation``` that we will be using needs a target as parameter. This target describes what the location designator is for, this could either be a pose or object that the robot should be able to see or reach.

In this case we only want poses where the robot can be placed, this is the default behaviour of the location designator which we will be extending later. 

In [2]:
from pycram.designators.location_designator import CostmapLocation

target = kitchen.get_pose()

location_description = CostmapLocation(target)

pose = location_description.resolve()

print(pose)

CostmapLocation.Location(pose=header: 
  seq: 0
  stamp: 
    secs: 1699448375
    nsecs: 569256305
  frame_id: "map"
pose: 
  position: 
    x: 0.32
    y: 0.46
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.8863025691598214
    w: -0.46310663556107684, reachable_arms=None)


## Reachable
Next we want to locations from where the robot can reach a specific point, like an object the robot should pick up. This can also be done with the ```CostmapLocation``` description, but this time we need to provide an additional argument. The additional argument is the robo which should be able to reach the pose. 

Since a robot is needed we will use the PR2 and use a milk as a target point for the robot to reach. The torso of the PR2 will be set to 0.2 since otherwise the arms of the robot will be too low to reach on the countertop.


In [3]:
pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")
pr2.set_joint_state("torso_lift_joint", 0.2)
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([1.3, 1, 0.9]))


Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [4]:
from pycram.designators.location_designator import CostmapLocation
from pycram.designators.object_designator import BelieveObject

target = BelieveObject(names=["milk"]).resolve()
robot_desig = BelieveObject(names=["pr2"]).resolve()

location_description = CostmapLocation(target=target, reachable_for=robot_desig)

print(location_description.resolve())

CostmapLocation.Location(pose=header: 
  seq: 0
  stamp: 
    secs: 1699448386
    nsecs: 194609165
  frame_id: "map"
pose: 
  position: 
    x: 0.52
    y: 0.94
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.03837651950358723
    w: 0.9992633500488202, reachable_arms=['left', 'right'])


As you can see we get a pose near the countertop where the robot can be placed without colliding with it. Furthermore, we get a list of arms with which the robot can reach the given object.

## Visibile
The ```CostmapLocation``` can also find position from which the robot can see a given object or location. This is very similar to how rechable locations are described, meaning we provide a object designator or a pose and a robot designator but this time we use the ```visible_for``` parameter.  

For this example we need the milk as well as the PR2, so if you did not spawn them during the previous location designator you can spawn them with the following cell.  

In [5]:
pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([1.3, 1, 0.9]))

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Exception in thread Thread-11Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
    self.run()
  File "/home/jdech/workspace/ros/src/pycram-1/src/pycram/bullet_world.py", line 467, in run
    b_pose = bulletworld_obj.get_pose()
  File "/home/jdech/workspace/ros/src/pycram-1

In [5]:
from pycram.designators.location_designator import CostmapLocation
from pycram.designators.object_designator import BelieveObject

target = BelieveObject(names=["milk"]).resolve()
robot_desig = BelieveObject(names=["pr2"]).resolve()

location_description = CostmapLocation(target=target, visible_for=robot_desig)

print(location_description.resolve())

CostmapLocation.Location(pose=header: 
  seq: 0
  stamp: 
    secs: 1699448390
    nsecs: 664081811
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.040000000000000036
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.4847685323929452
    w: 0.8746424812468178, reachable_arms=None)


## Semantic 
Semantic location designator are used to create location descriptions for semantic entities, like a table. An example of this is: You have a robot that picked up an object and should place it on a table. Semantic location designator then allows to find poses that are on this table.

Semantic location designator need an object from which the target entity is a part and the urdf link representing the entity. In this case we want a position on the kitchen island, so we have to provide the kitchen object designator since the island is a part of the kitchen and the link name of the island surface. 

For this example we need the kitchen as well as the milk. If you spawned them in one of the previous examples you don't need to execute the following cell.

In [8]:
kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")
milk = Object("milk", ObjectType.MILK, "milk.stl")

In [6]:
from pycram.designators.location_designator import SemanticCostmapLocation 
from pycram.designators.object_designator import BelieveObject

kitchen_desig = BelieveObject(names=["kitchen"]).resolve()
milk_desig = BelieveObject(names=["milk"]).resolve()

location_description = SemanticCostmapLocation(urdf_link_name="kitchen_island_surface", 
                                               part_of=kitchen_desig,
                                              for_object=milk_desig)

print(location_description.resolve())

SemanticCostmapLocation.Location(pose=header: 
  seq: 0
  stamp: 
    secs: 1699448395
    nsecs: 710832595
  frame_id: "map"
pose: 
  position: 
    x: -1.2074999952316285
    y: 1.019200015068054
    z: 0.9398907270729542
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.6339889056055381
    w: 0.7733421413379024)


## Location Designator as Generator
Location designator descriptions implement an iter method, so they can be used as generators which generate valid poses for the location described in the description. This can be useful if the first pose does not work for some reason. 

We will see this at the example of a location designator for visibility. For this example we need the milk, if you already have a milk spawned in you world you can ignore the following cell.

In [ ]:
milk = Object("milk", ObjectType.MILK, "milk.stl")

In [7]:
from pycram.designators.location_designator import CostmapLocation
from pycram.designators.object_designator import BelieveObject

target = BelieveObject(names=["milk"]).resolve()
robot_desig = BelieveObject(names=["pr2"]).resolve()

location_description = CostmapLocation(target=target, visible_for=robot_desig)

for pose in location_description:
    print(pose.pose)

header: 
  seq: 0
  stamp: 
    secs: 1699448399
    nsecs:  68569898
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.040000000000000036
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.4847685323929452
    w: 0.8746424812468178
header: 
  seq: 0
  stamp: 
    secs: 1699448399
    nsecs:  87555170
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.14
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.462503191327324
    w: 0.8866176165698721
header: 
  seq: 0
  stamp: 
    secs: 1699448399
    nsecs:  97248554
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.07999999999999996
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.47630463962315017
    w: 0.8792803251941107
header: 
  seq: 0
  stamp: 
    secs: 1699448399
    nsecs: 107382535
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.6
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.2897841486884303
    w: 0

header: 
  seq: 0
  stamp: 
    secs: 1699448399
    nsecs: 518246412
  frame_id: "map"
pose: 
  position: 
    x: 0.62
    y: 1.22
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.15581315965859366
    w: -0.9877865454019941
header: 
  seq: 0
  stamp: 
    secs: 1699448399
    nsecs: 527885913
  frame_id: "map"
pose: 
  position: 
    x: 0.62
    y: 1.2
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.14253858850979537
    w: -0.9897892456405228
header: 
  seq: 0
  stamp: 
    secs: 1699448399
    nsecs: 537669181
  frame_id: "map"
pose: 
  position: 
    x: 0.62
    y: 1.18
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.12902474983213136
    w: -0.9916413736481329
header: 
  seq: 0
  stamp: 
    secs: 1699448399
    nsecs: 546905040
  frame_id: "map"
pose: 
  position: 
    x: 0.62
    y: 1.16
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.11528840285654066
    w: -0.9933320613806785
header: 
  seq: 0
  stamp: 
    secs: 1699448399
    nsec

    w: -0.9469969274744021
header: 
  seq: 0
  stamp: 
    secs: 1699448399
    nsecs: 992797851
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 1.48
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.3310069414355005
    w: -0.9436283191604178
header: 
  seq: 0
  stamp: 
    secs: 1699448400
    nsecs:   3100872
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 1.5
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.3404252637530187
    w: -0.9402715776831115
header: 
  seq: 0
  stamp: 
    secs: 1699448400
    nsecs:  12321233
  frame_id: "map"
pose: 
  position: 
    x: 0.62
    y: 0.6599999999999999
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.2297529205473612
    w: 0.9732489894677302
header: 
  seq: 0
  stamp: 
    secs: 1699448400
    nsecs:  24640321
  frame_id: "map"
pose: 
  position: 
    x: 0.38
    y: 0.36
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.2992447144182441
    w: 0.9541763992536

header: 
  seq: 0
  stamp: 
    secs: 1699448400
    nsecs: 430586338
  frame_id: "map"
pose: 
  position: 
    x: 0.62
    y: 0.5800000000000001
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.2731318387159465
    w: 0.9619766102560116
header: 
  seq: 0
  stamp: 
    secs: 1699448400
    nsecs: 440135240
  frame_id: "map"
pose: 
  position: 
    x: 0.62
    y: 0.56
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.2832223886346635
    w: 0.9590542625816725
header: 
  seq: 0
  stamp: 
    secs: 1699448400
    nsecs: 449331283
  frame_id: "map"
pose: 
  position: 
    x: 0.32000000000000006
    y: 1.52
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.2415067072620041
    w: -0.970399150013779
header: 
  seq: 0
  stamp: 
    secs: 1699448400
    nsecs: 458734035
  frame_id: "map"
pose: 
  position: 
    x: 0.32000000000000006
    y: 1.5
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.2337072209964503
    w: -0.9723070167668834
header: 
  seq: 0
  st

    w: 0.9631193673564087
header: 
  seq: 0
  stamp: 
    secs: 1699448400
    nsecs: 941771745
  frame_id: "map"
pose: 
  position: 
    x: 0.64
    y: 0.62
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.25824169214694126
    w: 0.9660803426408615
header: 
  seq: 0
  stamp: 
    secs: 1699448400
    nsecs: 960236549
  frame_id: "map"
pose: 
  position: 
    x: 0.64
    y: 0.6599999999999999
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.23561137872925125
    w: 0.9718473533499494
header: 
  seq: 0
  stamp: 
    secs: 1699448400
    nsecs: 969287157
  frame_id: "map"
pose: 
  position: 
    x: 0.7200000000000001
    y: 1.26
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.2091538614095483
    w: -0.9778827446363267
header: 
  seq: 0
  stamp: 
    secs: 1699448400
    nsecs: 978359460
  frame_id: "map"
pose: 
  position: 
    x: 0.7200000000000001
    y: 1.28
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.22298919663900926
    w: -0.9748209159

...

header: 
  seq: 0
  stamp: 
    secs: 1699448402
    nsecs: 993033885
  frame_id: "map"
pose: 
  position: 
    x: 0.4600000000000001
    y: 1.72
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.3469462477349364
    w: -0.9378850149046248
header: 
  seq: 0
  stamp: 
    secs: 1699448403
    nsecs:   3197431
  frame_id: "map"
pose: 
  position: 
    x: 0.68
    y: 0.19999999999999996
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.440129716426979
    w: 0.8979342028890018
header: 
  seq: 0
  stamp: 
    secs: 1699448403
    nsecs:  22138833
  frame_id: "map"
pose: 
  position: 
    x: 0.68
    y: 0.14
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.4556305656033062
    w: 0.8901689658081837
header: 
  seq: 0
  stamp: 
    secs: 1699448403
    nsecs:  31298160
  frame_id: "map"
pose: 
  position: 
    x: 0.68
    y: 0.12
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.4604598069141401
    w: 0.8876805541503052
header: 
  seq: 0
  stamp: 
    secs: 

    w: -0.9888829578676007
header: 
  seq: 0
  stamp: 
    secs: 1699448403
    nsecs: 434596300
  frame_id: "map"
pose: 
  position: 
    x: 0.26
    y: 1.28
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.13111872764660726
    w: -0.9913666724579432
header: 
  seq: 0
  stamp: 
    secs: 1699448403
    nsecs: 444035291
  frame_id: "map"
pose: 
  position: 
    x: 0.26
    y: 1.26
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.12218326369570462
    w: -0.992507556682903
header: 
  seq: 0
  stamp: 
    secs: 1699448403
    nsecs: 454012870
  frame_id: "map"
pose: 
  position: 
    x: 0.26
    y: 1.24
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.11315653826752582
    w: -0.9935771725675414
header: 
  seq: 0
  stamp: 
    secs: 1699448403
    nsecs: 463644504
  frame_id: "map"
pose: 
  position: 
    x: 0.26
    y: 1.22
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.10404397359063852
    w: -0.994572697976106
header: 
  seq: 0
  stamp: 
   

header: 
  seq: 0
  stamp: 
    secs: 1699448404
    nsecs:  73369741
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.9
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0824805315448933
    w: 0.9965926760297167
header: 
  seq: 0
  stamp: 
    secs: 1699448404
    nsecs:  82538843
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.88
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.09853761796664222
    w: 0.9951333266680702
header: 
  seq: 0
  stamp: 
    secs: 1699448404
    nsecs:  92144012
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.86
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.11436518320320817
    w: 0.9934387776158613
header: 
  seq: 0
  stamp: 
    secs: 1699448404
    nsecs: 109889507
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.84
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.1299327910859183
    w: 0.9915228034698058
header: 
  

header: 
  seq: 0
  stamp: 
    secs: 1699448404
    nsecs: 508428335
  frame_id: "map"
pose: 
  position: 
    x: 0.74
    y: 1.34
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.26945545529666975
    w: -0.9630128543331415
header: 
  seq: 0
  stamp: 
    secs: 1699448404
    nsecs: 526400804
  frame_id: "map"
pose: 
  position: 
    x: 0.74
    y: 1.3599999999999999
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.281799152432585
    w: -0.9594734168742127
header: 
  seq: 0
  stamp: 
    secs: 1699448404
    nsecs: 575167417
  frame_id: "map"
pose: 
  position: 
    x: 0.74
    y: 1.42
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.316227766016838
    w: -0.9486832980505138
header: 
  seq: 0
  stamp: 
    secs: 1699448404
    nsecs: 591518163
  frame_id: "map"
pose: 
  position: 
    x: 0.74
    y: 1.44
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.326865156584162
    w: -0.945070986440284
header: 
  seq: 0
  stamp: 
    secs: 1699448404


header: 
  seq: 0
  stamp: 
    secs: 1699448405
    nsecs: 380416393
  frame_id: "map"
pose: 
  position: 
    x: 0.8
    y: 1.12
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.11750042131729284
    w: -0.993072832671531
header: 
  seq: 0
  stamp: 
    secs: 1699448405
    nsecs: 389935970
  frame_id: "map"
pose: 
  position: 
    x: 0.8
    y: 1.1
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.09853761796664212
    w: -0.9951333266680702
header: 
  seq: 0
  stamp: 
    secs: 1699448405
    nsecs: 399132251
  frame_id: "map"
pose: 
  position: 
    x: 0.8
    y: 1.08
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.07924445748219482
    w: -0.9968552131369695
header: 
  seq: 0
  stamp: 
    secs: 1699448405
    nsecs: 408757448
  frame_id: "map"
pose: 
  position: 
    x: 0.8
    y: 1.06
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.059678980860425196
    w: -0.9982176211846098
header: 
  seq: 0
  stamp: 
    secs: 1699448405
    nsecs: 4

## Accessing Locations
Accessing describes a location from which the robot can open a drawer. The drawer is specified by a ObjetcPart designator which describes the handle of the drawer.

At the moment this location designator only works in the apartment environment, so please remove the kitchen if you spawned it in a previous example. Furthermore, we need a robot so we also spawn the pr2 if it isn't spawned already.

In [8]:
kitchen.remove()

In [9]:
apartment = Object("apartment", ObjectType.ENVIRONMENT, "apartment.urdf")

Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]
Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]


In [10]:
pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")
pr2.set_joint_state("torso_lift_joint", 0.25)

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [10]:
from pycram.designators.object_designator import *
from pycram.designators.location_designator import *

apartment_desig = BelieveObject(names=["apartment"])
handle_desig = ObjectPart(names=["handle_cab10_t"], part_of=apartment_desig.resolve())
robot_desig = BelieveObject(names=["pr2"])

access_location = AccessingLocation(handle_desig.resolve(), robot_desig.resolve()).resolve()
print(access_location.pose)

header: 
  seq: 0
  stamp: 
    secs: 1699448422
    nsecs: 547766208
  frame_id: "map"
pose: 
  position: 
    x: 1.8074915790557862
    y: 2.7473597526550293
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.5893608715092853
    w: -0.8078698924541103
